In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB,CategoricalNB

# 8.1

In [ ]:
df1 = pd.read_csv('bank-full-encoded.csv')

In [ ]:
df1

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,40,4,1,2,0,3036,1,0,2,4,8,261,0,0,0,3,0
1,26,9,2,1,0,945,1,0,2,4,8,151,0,0,0,3,0
2,15,2,1,1,0,918,1,1,2,4,8,76,0,0,0,3,0
3,29,1,1,3,0,2420,1,0,2,4,8,92,0,0,0,3,0
4,15,11,2,3,0,917,0,0,2,4,8,198,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,33,9,1,2,0,1741,0,0,0,16,9,975,2,0,0,3,1
45207,53,5,0,0,0,2639,0,0,0,16,9,456,1,0,0,3,1
45208,54,5,1,1,0,5455,0,0,0,16,9,1116,4,181,3,2,1
45209,39,1,1,1,0,1584,0,0,1,16,9,508,3,0,0,3,0


## a

In [ ]:
x, y = df1.values[:,:-1], df1.values[:,-1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

## b

In [ ]:
model1 = GaussianNB().fit(x_train, y_train)

In [ ]:
#wyliczanie AUC
y_pred = model1.predict_proba(x_test)
#należy wziąć y dla wartości true (positive class)
y_pred1 = y_pred[:,1]
print(roc_auc_score(y_test, y_pred1))

0.8178964182392834


## c - proces dyskretyzacji

Nowa funkcj KBinsDiscretizer - podajemy ilość kubełków, to bd ilość zmiennych dyskretnych, możemy je zakodować za pomocą nr. kubełka (encode = 'ordinal') lub w systemie binarnym ('one-hot-encoding'). Strategia pozwala nam wybrać jak dzielimy czy długość przedziału ('uniform') czy ta sama ilość punktów ('quantile') jeszcze możliwe KNN ('kmeans')

In [ ]:
discretizer = KBinsDiscretizer(n_bins = 10, strategy = 'uniform', encode = 'ordinal')

ColumnTransformer - służy do przeprowadzania transformacji na kolumnach, możemy go nazwać, żeby się potem do niego odwoływać, mówimy też na które kolumny chcemy działać, remainder= 'passthrough' pozwala nam zatrzymać kolumny, które nie podelegały transformacji

In [ ]:
# zwróćmy uwagę na to, że nazwe, jakie transforme i na które kolumny podajemy w liście bo tyczy się to jednego hiperparametru 'transformers'
transformer = ColumnTransformer([("dyskretyzacja", discretizer, [0]+[5]+[9]+ list(range(11,15)))], remainder = 'passthrough')

In [ ]:
cnb = CategoricalNB()

Budowanie Pipelineu!!!

In [ ]:
pipe = Pipeline(steps = [('transformer_dyskretyzacja', transformer),('cnb', cnb)])

In [ ]:
model12 = pipe.fit(x_train, y_train)

In [ ]:
porb =model12.predict_proba(x_test)
roc_auc_score(y_test, porb[:,1])

0.8738691441120774

In [ ]:
disc = KBinsDiscretizer(n_bins = 10,strategy = 'uniform',encode = 'ordinal') # definiujemy funkcję która przerobi nam zmienne ciągłe na dyskretne
                                                                             # zmienna ciągła zostanie zmieniona na dyskretną o 10 poziomach 
                                                                             # (argument n_bins) do pierwszego koszyka wpadną najmniejsze wartości
                                                                             # do ostatniego największe. Argument strategy jest używany do określenia 
                                                                             # w jaki sposób są tworzone przedziały ('uniform' oznacza, że przedziały
                                                                             # mają mieć równą długość). encode oznacza w jaki sposób są kodowane 
                                                                             # utworzone zmienne dykretne ('ordinal' oznacza, że mamy jedną kolumnę 
                                                                             # gdzie dla każdej obserwacji jej wartość to numer koszyka 
                                                                             # do którego trafiła wartość przekształcanej zmiennej ciągłej).  
colt = ColumnTransformer([("disc",disc,[0]+[5]+[9]+list(range(11,15)))],remainder = 'passthrough') # ColumnTransformer to funkcja gdzie określamy
                                                                                                   # które kolumny mają być potraktowane konkretnym
                                                                                                   # przekształceniem, np. tutaj traktujemy
                                                                                                   # kolumnę o indeksie 0,5,9 i od 11 do 14 
                                                                                                   # KBinsDiscretizerem zdefiniowanym wcześniej,
                                                                                                   # remainder = 'passthrough' oznacza, że kolumny
                                                                                                   # których nie przekształcaliśmy są brane dalej
                                                                                                   # nieprzekształcone. Domyślnie remainder = 'drop'
                                                                                                   # oznacza, że nie bierzemy dalej kolumn,
                                                                                                   # które nie były przekształcane.
cnb = CategoricalNB() #kategoryczny naiwny bayes.
pipe = Pipeline(steps = [("colt",colt),("cnb",cnb)]) # tworzymy Pipeline, najpierw przekształcamy kolumny za pomocą colt, a następnie dopasowujemy
                                                     # klasyfikator cnb
pipe.fit(X_train,y_train) #fit w pipelinie oznacza, że robimy fit_transform na kolumnach ciągłych tj. liczymy granice przedziałów 
                          #na zbiorze treningowym dla kolumn ciągłych (za to odpowiada w fit_transform część fit) nastepnie mając granice
                          #przedziałów dyskretyzujemy kolejne kolumny (za to odpowiada część transform). Na tak przerobionym zbiorze danych 
                          #fitujemy klasyfikator CategoricalNB.
cnb_probs = pipe.predict_proba(X_test) #predict_proba w Pipelinie oznacza, że najpierw robimy transform na ciągłych kolumnach X_test 
                                       #(tj. dyskretyzujemy ciągłe kolumny X_test za pomocą przedziałów o granicach wyznaczonych za pomocą 
                                       # zbioru treningowego (bo na nim odbywał się fit_transform)). Na przetransformowanym zbiorze testowym
                                       # CategoricalNB dokonuje predict_proba
cnb_probs = cnb_probs[:, 1]
cnb_auc = roc_auc_score(y_test, cnb_probs) #liczymy AUC
print(cnb_auc)

# 9.2

In [ ]:
df2 = pd.read_csv('pima.csv')

In [ ]:
df2

,times,glucose,blood,skin,insulin,BMI,pedigree,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


## a

In [ ]:
x, y = df2.values[:,:-1], df2.values[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

## b

### metoda zastępowania danych za pomocą średniej

In [ ]:
# taki prosty sposob z poprzednich zajec
x_train2 = x_train.copy()
x_test2 = x_test.copy()
kolumna_blood = x_train2[:,2]
kolumna_blood[kolumna_blood == 0] = np.nan
x_train2[:,2] = SimpleImputer(strategy = 'mean').fit_transform(np.reshape(kolumna_blood,(-1,1))).flatten()

In [ ]:
# z uzyciem pipeline'u
x_train_pipeline = x_train.copy()
x_test_pipeline = x_test.copy()
x_train_pipeline[:,2][x_train_pipeline[:,2] == 0] = np.nan
x_test_pipeline[:,2][x_test_pipeline[:,2]==0] = np.nan

#definicja pipeline
imputer = SimpleImputer()
tree = DecisionTreeClassifier()
pipe2 = Pipeline(steps = [('imp', imputer), ('tree', tree)])

#nowa część pipeline grid!!! - pamietajmy, że miedzy transformerem a hiperparametrem o ktory nam chodzi musi byc podwojne podkreslenie
param_grid = {'imp__strategy':['mean', 'constant'],
              'imp__fill_value': [50], 
              "tree__max_depth" : [6, 7, 8, 9, 10, 11, 12],
              "tree__min_samples_leaf" : [4,5,6,7,8,9,10]}
        
# siatka przesukiwan
search = GridSearchCV(pipe2, param_grid, n_jobs = -1)
search.fit(x_train_pipeline, y_train)
print(search.best_params_)

{'imp__fill_value': 50, 'imp__strategy': 'constant', 'tree__max_depth': 7, 'tree__min_samples_leaf': 8}


## d dokładość na zbiorze testowym

In [ ]:
print(search.predict(x_test2))
#dokladność
print(search.score(x_test2, y_test))

[1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.
 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
0.7532467532467533


rudaś

In [ ]:
param_grid= {"simp__strategy":["mean","constant"],
             "simp__fill_value":[50],
             "tree__max_depth":[6,7,8,9,10,11,12],
             "tree__min_samples_leaf":[4,5,6,7,8,9,10]} #rozważamy siatkę parametrów dla niektórych argumentów simple imputera i drzewa,
                                                        #w gridsearchu będzie rozważona każda możliwa kombinacja parametrów w siatce. 
                                                        #Aby określić o jakie parametry konkretnie nam chodzi wpisujemy nazwę części pipeline'a,
                                                        #która zawiera dany parametr i po podwójnym podkreśleniu nazwę parametru danej metody.

search = GridSearchCV(pipe,param_grid,n_jobs=-1) #wyszukujemy za pomocą kroswalidacji 5-krotnej optymalny zestaw parametrów
search.fit(X_train2,y_train) #fitujemy gridsearcha celem znalezienia najlepszego zestawu parametrów (dla każdego układu parametrów robimy 
                             #kroswalidację. W danym obrocie pętli kroswalidacyjnej na części przewidzianej do uczenia robimy 
                             #fit_transform na simple imputerze i fit na modelu i liczymy dokładność na części walidacyjnej. 
                             #Uśredniamy dokładności po obrotach pętli kroswalidacyjnej i znajdujemy ten układ parametrów, gdzie 
                             #tak uśredniona dokladność jest największa).
print(search.best_params_) #printujemy najlepszy zestaw parametrów
search.predict(X_test2) # na X_test2 robimy transforma przy użyciu SipleImputera z optymalnymi parametrami znalezionymi w fit 
                        # we wcześniejszej linijce. Następnie na przetransformowanym zbiorze danych robimy predicta przy użyciu drzewa 
                        # z optymalnymi parametrami znalezionymi w fit we wcześniejszej linijce.
print(search.score(X_test2,y_test)) # podobnie jak linijkę temu, tylko teraz zamiast na końcu na optymalnym drzewie robić predicta, 
                                    # liczymy dokładność

tree = DecisionTreeClassifier() #dla porównania zwykłe drzewo z domyślnymi parametrami
tree.fit(X_train,y_train)
print(tree.score(X_test,y_test)) #nasz optymalny pipeline jest lepszy od zwykłego drzewa.

# 9.3

In [ ]:
df3 = pd.read_csv('SAheart2.csv')

In [ ]:
df3

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,NaN,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,NaN,3.50,27.78,Present,60,25.99,57.34,49,1
...,...,...,...,...,...,...,...,...,...,...
457,214,0.40,5.98,31.72,Absent,64,28.45,0.00,58,0
458,182,4.20,4.41,32.10,Absent,52,28.61,18.72,52,1
459,108,NaN,1.59,15.23,Absent,40,20.09,26.64,55,0
460,118,5.40,11.61,30.79,Absent,64,27.35,23.97,40,0


## a

In [ ]:
x3, y3 = df3.values[:,:-1], df3.values[:,-1]
x_train3, x_test3, y_train3, y_test3 = train_test_split(x3, y3, test_size = 0.3)

In [ ]:
y_train3 = y_train3.astype(int)
t_test3 = y_test3.astype(int)

In [ ]:
np.unique(y_train3)

array([0, 1])

##b

In [ ]:
#Zad 9.3
#a
url = 'SAheart2.csv'
dataframe = pandas.read_csv(url,header=0)
data = dataframe.values
print(dataframe)
X, y = data[:,:-1], data[:, -1]
y = y.astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.7,test_size = 0.3) #ten zbiór danych zawiera braki danych, 
                                                                                       #które musimy wypełnić

     sbp  tobacco    ldl  adiposity  famhist  typea  obesity  alcohol  age  \
0    160    12.00   5.73      23.11  Present     49    25.30    97.20   52   
1    144     0.01   4.41      28.61   Absent     55    28.87     2.06   63   
2    118     0.08   3.48      32.28      NaN     52    29.14     3.81   46   
3    170     7.50   6.41      38.03  Present     51    31.99    24.26   58   
4    134      NaN   3.50      27.78  Present     60    25.99    57.34   49   
..   ...      ...    ...        ...      ...    ...      ...      ...  ...   
457  214     0.40   5.98      31.72   Absent     64    28.45     0.00   58   
458  182     4.20   4.41      32.10   Absent     52    28.61    18.72   52   
459  108      NaN   1.59      15.23   Absent     40    20.09    26.64   55   
460  118     5.40  11.61      30.79   Absent     64    27.35    23.97   40   
461  132     0.00   4.82      33.41  Present     62    14.70     0.00   46   

     chd  
0      1  
1      1  
2      0  
3      1  
4      1

In [ ]:
imputer = SimpleImputer(strategy = "most_frequent")
encoder = OneHotEncoder()
imputer_ciagle = SimpleImputer(strategy ="mean")
algorytm = LogisticRegression(penalty = "none",max_iter=10000)
# na poczatku zmienne dyskretne 
dys = Pipeline(steps = [('usupelnianie_brakow', imputer), ('encoder', encoder)])

# ale uwaga musimy
trans = ColumnTransformer([("dyskrestne", dys, [4]),
                          ("ciagle", imputer_ciagle, list(range(0,4))+list(range(5,9)))])

pip2 = Pipeline(steps =[('transf_kolumn', trans), ('model', algorytm)])

In [ ]:
#b
imp_con = SimpleImputer() #simple imputer, który będziemy używać do zmiennych ciągłych (domyślnie wypełnia braki danych średnimi) 
imp_dis = SimpleImputer(strategy = 'most_frequent') #simple imputer, który będziemy używać do zmiennych dyskretnych (zatępujemy braki danych
                                                    #najczęściej powtarzającą się wartością)
enc_dis = OneHotEncoder(sparse = False,drop='first') #przerabiamy kolumnę dystkretną na k-1 kolumn 0-1kowych, gdzie k to liczba unikatowych wartości
                                                     #przyjmowanych przez daną kolumnę. 
pipe_dis = Pipeline(steps=[("imputer",imp_dis),("encoder",enc_dis)]) #robimy pipeline'a składającego się z imputera i encodingu 
                                                                     #(najpierw uzupełniamy braki danych dla zmiennych dyskretnych, a następnie 
                                                                     # wykonujemy one-hot-encoding)
colt = ColumnTransformer([("continous",imp_con,list(range(0,4))+list(range(5,9))),
                          ("discrete",pipe_dis,[4])]) #tworzymy column transformera, gdzie kolumnę dyskretną traktujemy wcześniejszym pipelinem,
                                                      #a kolumny ciągłe simple imputerem imp_con
m = LogisticRegression(penalty = "none",max_iter=10000) #definiujemy model regresji logistycznej
pipe = Pipeline(steps=[("prepr",colt),("logistic",m)])#tworzymy pipeline, gdzie na pierwszym miejscu mamy cały preprocessing, a później model liniowy

## c

In [ ]:
pipe.fit(x_train3, y_train3)
pipe.score(x_test3, y_test3)

ValueError: ignored